# **SCRAPPING IDEALISTA**

#### **Importación de librerías**

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import time
import random
from itertools import cycle
import pandas as pd
import json
import matplotlib.pyplot as plt

### **DEFINICIÓN DE ENCABEZADOS PARA SIMULAR LAS CONSULTAS**

Al hacer web scraping, es importante simular una solicitud HTTP desde un navegador web para evitar que el sitio web detecte que se está realizando una extracción automatizada de datos. Los encabezados HTTP son una parte importante de una solicitud y pueden incluir información sobre el navegador, el dispositivo y la configuración del sistema utilizados para realizar la solicitud.

Los encabezados especificados en el código anterior simulan una solicitud desde un navegador web específico (Google Chrome en un sistema Windows). Incluyen información sobre el tipo de contenido aceptado, el idioma preferido, el control de caché, la configuración de seguridad, la plataforma y el agente de usuario utilizados. Esto ayuda a disimular la solicitud y evitar que el sitio web bloquee la extracción de datos. Además, estos headers también pueden ayudar a evitar problemas de codificación o compresión de la respuesta, al especificar el tipo de codificación aceptado.

In [2]:
headers = {
    "accept": 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    "accept-encoding": 'gzip, deflate, br',
    "accept-language": 'es-ES,es;q=0.9,en;q=0.8',
    "cache-control": 'max-age=0',
    "dnt": "1",
    "sec-ch-ua": '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
    "sec-ch-ua-mobile": '?0',
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": 'document',
    "sec-fetch-mode": 'navigate',
    "sec-fetch-site": 'same-origin',
    "sec-fetch-user": '?1',
    "upgrade-insecure-requests": '1',
    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
}

In [3]:
proxy = {"http": "http://36.91.203.101:8080", "http" : "http://103.156.17.44:8082"}

### **Solicitud al portal**

Hacemos una solicitud al portal de Idealista usando la librería **request** y su función **.get**.
Definimos la función **"request_piso"** que posteriormente usaremos para enviar solicitudes a todos los ids obtenidos de la zona. 
Además, usamos **"time.sleep"** para hacer solicitudes pausadas entre 1 y 3 segundos y así hacer consultas más humanizadas.

In [4]:
id_inmueble = "97949995"

In [5]:
def request_piso(id_inmueble, headers="",proxies=""):
    url = f"https://www.idealista.com/inmueble/{id_inmueble}/"
    r = requests.get(url, headers=headers, proxies=proxy)
    time.sleep(random.randint(1,3)*random.random())
    return r

In [6]:
r = request_piso(id_inmueble, headers=headers)

In [7]:
r

<Response [403]>

Han terminado bloqueando nuestro acceso por completo. No podemos scrapear actualmente datos.

### **Función para la extracción de información de la vivienda**

In [8]:
soup = bs(r.text, "html.parser")

In [9]:
def info_inmueble():
    
    scripts = soup.find_all("script")
    
    for script in scripts:
        if "var config" in str(script):
            break
    latitude = float(str(script).split("latitude")[1].split(",")[0].split()[1].replace("'", ""))        
    longitude = float(str(script).split("latitude")[1].split(",")[1].strip().split()[1].replace("'", ""))
    
    scripts = soup.find_all("script")
    
    for script in scripts:
        if "var utag_data" in str(script):
            break
    
    data = json.loads(str(script).split("var utag_data =")[1].split(";")[0].strip())
    
    titulo = soup.find("h1").text.strip()
    localizacion = soup.find("span",{"class":"main-info__title-block"}).find("span").text.strip()
    price = data["ad"]["price"]
    energy_certification = data["ad"]["energyCertification"]["type"]
    basic_characteristics = data["ad"]["characteristics"]
    room_number = data["ad"]["characteristics"]["roomNumber"]
    bath_number = data["ad"]["characteristics"]["bathNumber"]
    
    try:
        has_garden = data["ad"]["characteristics"]["hasGarden"]
    except:
        has_garden = None
        
    try:
        has_terrace = data["ad"]["characteristics"]["hasTerrace"]
    except:
        has_terrace = None
        
    try:
        has_parking = data["ad"]["characteristics"]["hasParking"]
    except:
        has_parking = None
        
    try:
        has_swimmingpool = data["ad"]["characteristics"]["hasSwimmingPool"]
    except:
        has_swimmingpool = None
        
    try:
        has_lift = data["ad"]["characteristics"]["hasLift"]
    except:
        has_lift = None
        
    constructed_area = data["ad"]["characteristics"]["constructedArea"]
    is_new_development = data["ad"]["condition"]["isNewDevelopment"]
    is_needs_renovating = data["ad"]["condition"]["isNeedsRenovating"]
    is_goog_condition = data["ad"]["condition"]["isGoodCondition"]
    
    lista = [titulo, localizacion, latitude, longitude, price, energy_certification, basic_characteristics,
            room_number, bath_number,has_garden, has_terrace, has_parking, has_swimmingpool, has_lift, constructed_area,
            is_new_development, is_needs_renovating, is_goog_condition]
    
    time.sleep(random.randint(1,3)*random.random())
    return lista

In [10]:
info_inmueble()

IndexError: list index out of range

### **Extracción de IDs de inmuebles**

A partir de una **variable de entrada "busqueda"** en la que establecemos la zona de interés, en este caso **"churriana malaga"** se procede a extraer todos los ids identificados a partir de un **bucle while true** que permite scrapear todas las viviendas de todas las paǵinas existentes hasta el último inmueble. Los ids se van **almacenando en una lista usando la función .append**

In [13]:
busqueda = "churriana malaga"
busqueda = busqueda.replace(" ","_")

x = 1
ids = []

while True:
    url = f"https://www.idealista.com/buscar/venta-viviendas/{busqueda}/pagina-{x}.htm"
    
    r = requests.get(url, headers = headers)
    soup = bs(r.text, "html.parser")
    
    pagina_actual = int(soup.find("main", {"class":"listing-items"}).find("div", {"class":"pagination"}).find("li", {"class":"selected"}).text)
    
    if x == pagina_actual:
        articles = soup.find("main", {"class":"listing-items"}).find_all("article")
    
    else:
        break
        
    x = x+1
        
    for article in articles:
        id_inmueble = article.get("data-adid")
        
        ids.append(id_inmueble)
        
    time.sleep(random.randint(1,3)*random.random())

In [14]:
len(ids)

342

Guardamos la lista de ids **en un archivo .csv** 

In [15]:
columns_ids = ["id"]
df_ids = pd.DataFrame(ids, columns= columns_ids)

df_ids.to_csv('ids_churriana-malaga.csv')

In [8]:
df = pd.read_csv('ids_churriana-malaga.csv')

In [9]:
df.head()

,Unnamed: 0,id
0,0,98811885
1,1,95931901
2,2,98750560
3,3,98942399
4,4,97949995


In [10]:
list_ids = df["id"].tolist()

In [13]:
len(list_ids)

342

### **EXTRACCIÓN CARACTERÍSTICAS INMUEBLES**

A partir de un buble for recorreremos la lista de ids generada para Churriana sobre la que aplicaremos la función definida previamente para hacer solicitudes de consulta al portal "request_piso" y la función "info_inmuebles" para obtener todas las características de cada id.

Guardamos en una lista toda la información de los inmuebles.

El programa cuándo realiza cierto volumen de consultas se para y nos da el siguiente error: IndexError: list index out of range. Esto se debe a que Idealista identifica solicitudes sospechosas y nos bloquea el asceso. 

Se consigue recabar información del 55% del total de inmuebles, por lo que se decide seguir con el análisis usando dichos datos.


In [30]:
lista_info_inmuebles = []

for id_inmueble in ids:
    r = request_piso(id_inmueble= id_inmueble, headers=headers)
    soup = bs(r.text, "html.parser")
    info_inmueble1 = info_inmueble()
    lista_info_inmuebles.append(info_inmueble1)
    time.sleep(random.randint(1,3)*random.random())

print(id_inmueble)

IndexError: list index out of range

### **CREACIÓN DATASET**

En primer lugar **definimos las columnas** que va a tener nuestro conjunto de datos y posteriormente a partir de la librería **pandas** generamos nuestro dataframe usando la información de la lista de inmuebles y las columnas definidas. 

Por último guardamos nuestros datos en un .csv.

In [24]:
columns = ["titulo", "localizacion", "latitude", "longitude", "price", "energy_certification", "basic_characteristics",
            "room_number", "bath_number", "has_garden", "has_terrace", "has_parking", "has_swimmingpool", "has_lift", "constructed_area",
            "is_new_development", "is_needs_renovating", "is_goog_condition"]

In [25]:
df_inmuebles = pd.DataFrame(lista_info_inmuebles, columns= columns)

In [26]:
df_inmuebles.head()

,titulo,localizacion,latitude,longitude,price,energy_certification,basic_characteristics,room_number,bath_number,has_garden,has_terrace,has_parking,has_swimmingpool,has_lift,constructed_area,is_new_development,is_needs_renovating,is_goog_condition
0,Chalet adosado en venta en Guadalmar,"Churriana, Málaga",36.666638,-4.464576,395000,inProcess,"{'roomNumber': '4', 'bathNumber': '3', 'hasPar...",4,3,1,1,1,1,None,194,0,0,1
1,Casa o chalet independiente en venta en calle ...,"Churriana-El Pizarrillo-La Noria-Guadalsol, Má...",36.663719,-4.503897,640000,inProcess,"{'roomNumber': '4', 'bathNumber': '2', 'hasPar...",4,2,1,1,1,1,None,310,0,0,1
2,Casa o chalet independiente en venta en calle ...,"Cortijo de Maza-Finca Monsalvez-El Olivar, Málaga",36.656894,-4.502930,567000,inProcess,"{'roomNumber': '5', 'bathNumber': '4', 'hasPar...",5,4,1,1,0,1,None,410,0,0,1
3,"Ático en venta en calle Bangladesh, 25","Churriana-El Pizarrillo-La Noria-Guadalsol, Má...",36.671190,-4.516900,375900,unknown,"{'roomNumber': '4', 'bathNumber': '2', 'hasLif...",4,2,1,1,1,1,1,236,1,0,0
4,"Piso en venta en calle Bangladesh, 25","Churriana-El Pizarrillo-La Noria-Guadalsol, Má...",36.671190,-4.516900,249900,unknown,"{'roomNumber': '2', 'bathNumber': '2', 'hasLif...",2,2,0,1,1,1,1,150,1,0,0
5,"Piso en venta en calle Bangladesh, 25","Churriana-El Pizarrillo-La Noria-Guadalsol, Má...",36.671190,-4.516900,230900,unknown,"{'roomNumber': '2', 'bathNumber': '2', 'hasLif...",2,2,0,1,1,1,1,99,1,0,0
6,Casa o chalet en venta en Cortijo de Maza-Finc...,"Churriana, Málaga",36.651026,-4.493962,795000,c,"{'roomNumber': '3', 'bathNumber': '4', 'hasPar...",3,4,1,1,1,1,None,702,0,1,0
7,Chalet adosado en venta en Churriana-El Pizarr...,"Churriana, Málaga",36.670427,-4.507101,235000,e,"{'roomNumber': '3', 'bathNumber': '2', 'hasPar...",3,2,0,0,1,1,None,149,0,0,1
8,Casa o chalet independiente en venta en Cortij...,"Churriana, Málaga",36.654335,-4.490218,690000,inProcess,"{'roomNumber': '5', 'bathNumber': '4', 'hasPar...",5,4,1,1,1,1,None,649,0,0,1
9,Chalet adosado en venta en Churriana-El Pizarr...,"Churriana, Málaga",36.659422,-4.496192,371000,inProcess,"{'roomNumber': '3', 'bathNumber': '3', 'hasPar...",3,3,None,1,1,1,None,152,0,0,1


In [28]:
df_inmuebles.to_csv('info-churriana-malaga.csv')

Ya tenemos nuestro conjunto de datos en bruto. El siguiente paso será explorar los datos, hacer una limpieza, y preparar los mismos para la aplicación de los modelos de Maching Learning que nos permitiŕa hacer predicciones del precio de la viviendas de la zona de estudio.